In [1]:
# task2.py を逐次実行してみて中身を把握する

import sys
import os

import numpy as np
import argparse
import textwrap


import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

sys.path.append('/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/')

from dcase_framework.application_core import BinarySoundEventAppCore
from dcase_framework.parameters import ParameterContainer
from dcase_framework.utils import *
from dcase_framework.features import FeatureContainer, FeatureRepository, FeatureExtractor, FeatureNormalizer, \
    FeatureStacker, FeatureAggregator, FeatureMasker


In [2]:
project_base=os.path.dirname(os.path.realpath('../../baseline/DCASE2017-baseline-system/applications/system/'))


params = ParameterContainer(
    project_base=project_base,
    path_structure={
        'feature_extractor': [
            'dataset',
            'feature_extractor.parameters.*'
        ],
        'feature_normalizer': [
            'dataset',
            'feature_extractor.parameters.*'
        ],
        'learner': [
            'dataset',
            'feature_extractor',
            'feature_stacker',
            'feature_normalizer',
            'feature_aggregator',
            'learner'
        ],
        'recognizer': [
            'dataset',
            'feature_extractor',
            'feature_stacker',
            'feature_normalizer',
            'feature_aggregator',
            'learner',
            'recognizer'
        ],
    }
)

params

{}

In [3]:
params.load(filename=project_base + '/parameters/task2.defaults.yaml')
params['active_set'] = 'dcase2017_gpu'
params.process() # 実行用のパラメータ構造に変える？


setup_logging(parameter_container=params['logging'])
#app.show_parameters()

/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/dcase_framework/files.py:379: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dict.update(self, yaml.load(infile))


In [4]:
class Task2AppCore(BinarySoundEventAppCore):
    pass

app = Task2AppCore(
    name='DCASE 2017::Detection of rare sound events / Baseline System',
    params=params,
    system_desc=params.get('description'),
    system_parameter_set_id=params.get('active_set'),
    setup_label='Development setup',
    log_system_progress=params.get_path('general.log_system_progress'),
    show_progress_in_console=params.get_path('general.print_system_progress'),
    use_ascii_progress_bar=params.get_path('general.use_ascii_progress_bar')
)

# app.initialize()

[I] DCASE 2017::Detection of rare sound events / Baseline System
[I] 


In [6]:
from tqdm import tqdm
from six import iteritems

fold_progress = tqdm(app._get_active_folds(),
                     desc='           {0:<15s}'.format('Fold '),
                     file=sys.stdout,
                     leave=False,
                     miniters=1,
                     disable=app.disable_progress_bar,
                     ascii=app.use_ascii_progress_bar)


for fold in fold_progress:
    if app.log_system_progress:
        app.logger.info('  {title:<15s} [{fold:d}/{total:d}]'.format(
            title='Fold',
            fold=fold,
            total=len(fold_progress))
        )

    event_labels = app.dataset.event_labels
    # Select only active events
    if app.params.get_path('learner.active_events'):
        event_labels = list(set(event_labels).intersection(app.params.get_path('learner.active_events')))

    event_progress = tqdm(
        event_labels,
        desc="           {0: >15s}".format('Event '),
        file=sys.stdout,
        leave=False,
        miniters=1,
        disable=app.disable_progress_bar,
        ascii=app.use_ascii_progress_bar
    )

    event_label = 'babycry'
    current_model_file = app._get_model_filename(
        fold=fold,
        path=app.params.get_path('path.learner'),
        event_label=event_label
    )
    
    # current_model_file は存在済みだが処理を続行
    
    feature_processing_chain = app.ProcessingChain()

    # Feature stacker
    feature_stacker = FeatureStacker(
        recipe=app.params.get_path('feature_stacker.stacking_recipe'),
        feature_hop=app.params.get_path('feature_stacker.feature_hop', 1)
    )
    feature_processing_chain.append(feature_stacker)



    # Feature normalizer
    feature_normalizer = None
    if app.params.get_path('feature_normalizer.enable'):
        # Load normalizers
        feature_normalizer_filenames = app._get_feature_normalizer_filename(
            fold=fold,
            path=app.params.get_path('path.feature_normalizer'),
            event_label=event_label
        )

        normalizer_list = {}
        for method, feature_normalizer_filename in iteritems(feature_normalizer_filenames):
            if os.path.isfile(feature_normalizer_filename):
                normalizer_list[method] = FeatureNormalizer().load(
                    filename=feature_normalizer_filename
                )

            else:
                message = '{name}: Feature normalizer not found [{file}]'.format(
                    name=app.__class__.__name__,
                    file=feature_normalizer_filename
                )

                app.logger.exception(message)
                raise IOError(message)

        feature_normalizer = app.FeatureNormalizer(feature_stacker.normalizer(
            normalizer_list=normalizer_list)
        )
        feature_processing_chain.append(feature_normalizer)

    # Feature aggregator
    feature_aggregator = None
    if app.params.get_path('feature_aggregator.enable'):
        feature_aggregator = FeatureAggregator(
            recipe=app.params.get_path('feature_aggregator.aggregation_recipe'),
            win_length_frames=app.params.get_path('feature_aggregator.win_length_frames'),
            hop_length_frames=app.params.get_path('feature_aggregator.hop_length_frames')
        )
        feature_processing_chain.append(feature_aggregator)

        # Data processing chain
        data_processing_chain = app.ProcessingChain()
        if app.params.get_path('learner.parameters.input_sequencer.enable'): # サンプルではNone 
            data_sequencer = app.DataSequencer(
                frames=app.params.get_path('learner.parameters.input_sequencer.frames'),
                hop=app.params.get_path('learner.parameters.input_sequencer.hop'),
                padding=app.params.get_path('learner.parameters.input_sequencer.padding'),
                shift_step=app.params.get_path('learner.parameters.temporal_shifter.step') if app.params.get_path('learner.parameters.temporal_shifter.enable') else None,
                shift_border=app.params.get_path('learner.parameters.temporal_shifter.border') if app.params.get_path('learner.parameters.temporal_shifter.enable') else None,
                shift_max=app.params.get_path('learner.parameters.temporal_shifter.max') if app.params.get_path('learner.parameters.temporal_shifter.enable') else None,
            )
            data_processing_chain.append(data_sequencer)

        # Data processor
        data_processor = app.DataProcessor(
            feature_processing_chain=feature_processing_chain,
            data_processing_chain=data_processing_chain,
        )

        # Collect training examples
        train_meta = app.dataset.train(fold, event_label=event_label)
        data = {}
        data_filelist = {}
        annotations = {}

        item_progress = tqdm(train_meta.file_list[::app.params.get_path('learner.file_hop', 1)],
                             desc="           {0: >15s}".format('Collect data '),
                             file=sys.stdout,
                             leave=False,
                             bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ',
                             disable=app.disable_progress_bar,
                             ascii=app.use_ascii_progress_bar)

        

        # Collect learning examples
        for item_id, audio_filename in enumerate(item_progress):
            if app.log_system_progress:
                app.logger.info(
                    '  {title:<15s} [{item_id:3s}/{total:3s}] {item:<20s}'.format(
                        title='Collect data ',
                        item_id='{:d}'.format(item_id),
                        total='{:d}'.format(len(item_progress)),
                        item=os.path.split(audio_filename)[-1])
                    )

            item_progress.set_postfix(file=os.path.splitext(os.path.split(audio_filename)[-1])[0])
            item_progress.update()

            # Get feature filenames
            feature_filenames = app._get_feature_filename(
                audio_file=audio_filename,
                path=app.params.get_path('path.feature_extractor')
            )

            if not app.params.get_path('learner.parameters.generator.enable'):
                # If generator is not used, load features now.
                # Do only feature processing here. Leave data processing for learner.
                feature_data, feature_length = data_processor.load(
                    feature_filename_dict=feature_filenames,
                    process_features=True,
                    process_data=False
                )
                data[audio_filename] = FeatureContainer(features=[feature_data])

            # Inject audio_filename to the features filenames for the raw data generator
            feature_filenames['_audio_filename'] = audio_filename
            data_filelist[audio_filename] = feature_filenames

            annotations[audio_filename] = train_meta.filter(filename=audio_filename)

        if app.log_system_progress:
            app.logger.info(' ')

        # Get learner
        learner = app._get_learner(
            method=app.params.get_path('learner.method'),
            class_labels=[event_label],
            data_processor=data_processor,
            feature_processing_chain=feature_processing_chain,
            feature_normalizer=feature_normalizer,
            feature_stacker=feature_stacker,
            feature_aggregator=feature_aggregator,
            params=app.params.get_path('learner'),
            filename=current_model_file,
            disable_progress_bar=app.disable_progress_bar,
            log_progress=app.log_system_progress,
            data_generators=app.DataGenerators if app.params.get_path('learner.parameters.generator.enable') else None,
        )

        # Get validation files from dataset
        validation_files = app.dataset.validation_files(fold, event_label=event_label)

        # Start learning
        learner.learn(
            data=data,
            annotations=annotations,
            data_filenames=data_filelist,
            validation_files=validation_files
        )

        # Save model
        learner.save()

        




           Fold           :   0%|          | 0/1 [00:00<?, ?it/s]



                    Event :   0%|          | 0/3 [00:00<?, ?it/s]




             Collect data :   0%|          | 0/491 




             Collect data :   0%|          | 0/491 




             Collect data :   0%|          | 1/491 




             Collect data :   0%|          | 2/491 




             Collect data :   1%|          | 3/491 




             Collect data :   1%|          | 3/491 




             Collect data :   1%|          | 4/491 




             Collect data :   1%|          | 5/491 




             Collect data :   1%|          | 6/491 




             Collect data :   1%|▏         | 7/491 




             Collect data :   1%|▏         | 7/491 




             Collect data :   2%|▏         | 8/491 




             Collect data :   2%|▏         | 9/491 




             Collect data :   2%|▏         | 10/491 




             Collect data :   2%|▏         | 10/491 




             Col

[D]   Validation
[D]     Event label          | Files (%)            
[D]     -------------------- + -------------------- 
[D]     -                    | 10.08 
[D]     babycry              | 10.29 
[D]  
[D]   Training items 	[661941]
[D]   Validation items 	[75050]
[D]   Keras
[D]     Backend 	[tensorflow]
[D]     BLAS library	[MKL]		(Threads[4], MKL_CBWR[COMPATIBLE])
[D]   Tensorflow
[D]     Device 		[gpu]
[D]   
[W] From /home/hirotoshi/.pyenv/versions/anaconda3-2019.10/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

[W] From /home/hirotoshi/.pyenv/versions/anaconda3-2019.10/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

[W] From /home/hirotoshi/.pyenv/versions/anaconda3-2019.10/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.ra






     Learn:   0%|          | 0/200 [00:00<?, ?it/s]

[W] From /home/hirotoshi/.pyenv/versions/anaconda3-2019.10/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

[W] From /home/hirotoshi/.pyenv/versions/anaconda3-2019.10/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:184: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

[W] From /home/hirotoshi/.pyenv/versions/anaconda3-2019.10/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

[W] From /home/hirotoshi/.pyenv/versions/anaconda3-2019.10/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

[W] From /home/hirotoshi/.pyenv/versions/anaconda3-2019.10/lib/python3.6/site-packages/keras/backe






     Learn:   0%|          | 0/200 [00:07<?, ?it/s, l_tra=0.114, l_val=0.101, m_tra=0.971, m_val=0.975]




     Learn:   0%|          | 1/200 [00:07<25:09,  7.58s/it, l_tra=0.114, l_val=0.101, m_tra=0.971, m_val=0.975]




     Learn:   0%|          | 1/200 [00:12<25:09,  7.58s/it, l_tra=0.087, l_val=0.092, m_tra=0.978, m_val=0.976]




     Learn:   1%|          | 2/200 [00:12<22:22,  6.78s/it, l_tra=0.087, l_val=0.092, m_tra=0.978, m_val=0.976]




     Learn:   1%|          | 2/200 [00:17<22:22,  6.78s/it, l_tra=0.078, l_val=0.082, m_tra=0.980, m_val=0.979]




     Learn:   2%|▏         | 3/200 [00:17<20:28,  6.24s/it, l_tra=0.078, l_val=0.082, m_tra=0.980, m_val=0.979]




     Learn:   2%|▏         | 3/200 [00:22<20:28,  6.24s/it, l_tra=0.074, l_val=0.082, m_tra=0.981, m_val=0.979]




     Learn:   2%|▏         | 4/200 [00:22<18:43,  5.73s/it, l_tra=0.074, l_val=0.082, m_tra=0.981, m_val=0.979]




     Learn:   2%|▏         | 4/200 [00:26<18:43,  5.73s/it, l_tra=0.070, l_

[I]   Stopping criteria met at epoch[127]
[I]     metric[val_binary_accuracy], patience[10]
[I]   







                                                                                                                 


           Fold           : 100%|██████████| 1/1 [10:21<00:00, 621.98s/it]



                                                     


                                                                          

In [30]:
feature_data = '/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/applications/system/task2/feature_extractor/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_57a55966f25a1b3722bd922a63c75875/sequence_mixture_devtrain_gunshot_499_540db42d3da254d22ee9e4ad0361cb00.cpickle'

feature_stacker = FeatureStacker(
    recipe=app.params.get_path('feature_stacker.stacking_recipe'),
    feature_hop=app.params.get_path('feature_stacker.feature_hop', 1)
)
feature_stacker.process(feature_data)


TypeError: string indices must be integers

In [45]:
train_meta # meta_data読み込んでる

[{'file': '/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/applications/data/TUT-rare-sound-events-2017-development/generated_data/mixtures_devtrain_0367e094f3f5c81ef017d128ebff4a3c/audio/mixture_devtrain_babycry_000_127e7b179bf1ce953739b7faa295cdc1.wav',
  'onset': 20.463606395727393,
  'offset': 23.50360639572739,
  'event_label': 'babycry',
  'event_onset': 20.463606395727393,
  'event_offset': 23.50360639572739},
 {'file': '/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/applications/data/TUT-rare-sound-events-2017-development/generated_data/mixtures_devtrain_0367e094f3f5c81ef017d128ebff4a3c/audio/mixture_devtrain_babycry_001_05d8cdf270e916e5f0c7806bcb65abe1.wav',
  'onset': 20.75749831062934,
  'offset': 22.89749831062934,
  'event_label': 'babycry',
  'event_onset': 20.75749831062934,
  'event_offset': 22.89749831062934},
 {'file': '/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/applications/data/TUT-rare-sound-ev

TypeError: 'NoneType' object is not subscriptable

In [60]:
data[audio_filename]



{'feat': [array([[-5.61218105e-03,  1.03808425e-01,  6.83329230e-02, ...,
           3.45209576e-01, -1.90595458e-01, -6.50209422e-01],
         [-5.61218105e-03,  1.03808425e-01,  6.83329230e-02, ...,
           1.03762035e+00,  3.89745731e-01,  7.32114193e-02],
         [-5.61218105e-03,  1.03808425e-01,  6.83329230e-02, ...,
           1.23333216e+00,  9.59806392e-01,  6.44027242e-01],
         ...,
         [ 6.08185288e-01,  1.08229439e+00,  1.93568565e+00, ...,
          -1.31877964e-01, -2.89353303e-04,  7.80843638e-03],
         [ 5.25859690e-01,  6.93980369e-01,  8.70915223e-01, ...,
          -1.31877964e-01, -2.89353303e-04,  7.80843638e-03],
         [ 5.21058093e-01,  4.70174320e-01,  9.32566827e-01, ...,
          -1.31877964e-01, -2.89353303e-04,  7.80843638e-03]])],
 'stat': None,
 'meta': {}}

In [64]:
import sys

sys.getsizeof(data)

18528

In [68]:
learner.default_parameters

{'win_length_seconds': 0.1,
 'hop_length_seconds': 0.02,
 'method': 'mlp',
 'scene_handling': 'scene-dependent',
 'parameters': {'seed': 5,
  'keras': {'backend': 'tensorflow',
   'backend_parameters': {'CNR': True,
    'device': 'gpu',
    'fastmath': True,
    'floatX': 'float32',
    'openmp': True,
    'optimizer': 'FAST_RUN',
    'threads': 4}},
  'model': {'config': [{'class_name': 'Dense',
     'config': {'units': 50,
      'kernel_initializer': 'uniform',
      'activation': 'relu'}},
    {'class_name': 'Dropout', 'config': {'rate': 0.2}},
    {'class_name': 'Dense',
     'config': {'units': 50,
      'kernel_initializer': 'uniform',
      'activation': 'relu'}},
    {'class_name': 'Dropout', 'config': {'rate': 0.2}},
    {'class_name': 'Dense',
     'config': {'units': 'CLASS_COUNT',
      'kernel_initializer': 'uniform',
      'activation': 'sigmoid'}}],
   'loss': 'binary_crossentropy',
   'metrics': ['binary_accuracy'],
   'optimizer': {'type': 'Adam'}},
  'training': {'bat

In [69]:
app.params.get_path('learner.method')

'mlp'

In [71]:
print(learner.model)

{}


In [7]:
app.params.get_path('feature_aggregator.win_length_frames')

5

In [10]:
from dcase_framework.learners import *
from dcase_framework.utils import get_class_inheritors, posix_path, check_pkg_resources

learner_list = get_class_inheritors(EventDetector)

In [11]:
learner_list

[dcase_framework.learners.EventDetectorGMM,
 dcase_framework.learners.EventDetectorGMMdeprecated,
 dcase_framework.learners.EventDetectorMLP,
 dcase_framework.learners.EventDetectorKerasSequential]

In [12]:
for learner_item in learner_list:
    learner = learner_item()
    if learner.method:
        app.Learners[learner.method] = learner_item

In [13]:
app.Learners

{'gmm': dcase_framework.learners.EventDetectorGMM,
 'gmm_deprecated': dcase_framework.learners.EventDetectorGMMdeprecated,
 'mlp': dcase_framework.learners.EventDetectorMLP,
 'keras_seq': dcase_framework.learners.EventDetectorKerasSequential}